# Testing the Functionality with Real Data!
04.08.2020
by jjr

---
Now, a real sample dataset from oil-under-ice will be loaded and a single packet will be processed with this DSP algorithm to ensure compatibility and proper operation.    

To streamline the process of debugging and testing these functions in the Jupyter Notebook environment, the supporting function definitions are provided first, followed by the DSP core functions that product the resultant matrices.  It was demonstrated in another notebook using these algorithms that the triggering logic is not sufficiently generalized to apply them to waveforms other than square-waves (as some of the distortions in the real-data will cause the trigger detection algorithm to fail and crash the processing application. 

First, the functions are defined for the DSP algorithms applied. 

---

## 1.  The DSP Algorithm Setup

In [1]:
#  module imports
import matplotlib.pyplot as plt
import numpy as np
import numpy.fft as fft
import numpy.linalg as linalg
import os

In [2]:
# Settings
VERBOSE = True
DEBUG = True
DEBUG_PLOT = True
ANALYSIS_PLOT = True

### 1A. Global Constants and Conversion Factor Definitions

In [3]:
# Global Constants for an Analysis
PACKET_NUMBER = 1
SAMPLE_PERIOD = 1  # seconds
PI = np.pi  

In [4]:
def convert_rads_to_hertz(frequency_domain):
    """
    :param frequency_domain: Resulting frequency domain from fft.rfftfreq function.
    """
    hz_per_rad_s = 2 * np.pi
    hertz_values = frequency_domain * hz_per_rad_s
    return hertz_values

In [5]:
def convert_hertz_to_rad_s(frequency_domain):
    """
    :param frequency_domain: Resulting frequency domain from fft.rfftfreq function.
    """
    hz_per_rad_s = 2 * np.pi
    rad_s_per_hz = 1 / hz_per_rad_s  # rads/Hz
    rad_s_values = frequency_domain * hz_per_rad_s
    return rad_s_values

### 1B. Data Import from Text File

In [6]:
def import_data(input_file_string):
    input_file_path = os.path.join(os.getcwd(), input_file)
    print(input_file_path)
    p_start_marker = f"${PACKET_NUMBER}\n"  # consider using a regex here instead for better precision/accuracy
    p_end_marker = "*"

    # This parsing method seems quite speedy with a bunch of lazy evaluation... 
    # interestingly, the next() method for generators will allow for iteration through 
    # ALL packets lazily. 
    with open(input_file_path, "r") as file:
        lines = file.readlines()
        start_generator = (index for index, line in enumerate(lines) if line.startswith(p_start_marker))
        end_generator = (index for index, line in enumerate(lines) if line.startswith(p_end_marker))
        raw_packet_list = lines[next(start_generator) + 10: next(end_generator)]
        
    raw_signals_matrix = np.genfromtxt(raw_packet_list, delimiter=",")
    

### 1C.  Derived Constants

In [7]:
def set_derived_constants(signals_matrix, PERIOD):
    # Everything seems in order here, which is good... 
    # One can also derive the array dimensions needed in the next steps from the input 
    # file alone with no other a priori knowledge with is also good. 
    N_SAMPLES = signals_matrix.shape[0]
    Q_CHANNELS = signals_matrix.shape[-1]
    F_S = np.divide(N_SAMPLES, PERIOD)
    TAU = np.divide(1, F_S)
    
    # verbose testprint for debugging:
    if VERBOSE:
        print(f"Array loaded for packet {PACKET_NUMBER}: Dimensions ({N_SAMPLES} X {Q_CHANNELS})")
        print(f"Sampling Frequency: {F_S} [S/s] , sampled over {PERIOD} seconds)")
    
    return N_SAMPLES, Q_CHANNELS, F_S, TAU

### 1D. Filter Design and Definition

In [8]:
### Filtering
def butterworth_digital_lpf(sig, n_samples, f_sample, f_order, f_cut, analysis_plot=False):
    """
    .. function:: butterworth_analog_lpf
    .. description::
    :param sig:
    :param order:
    :param f_cut:
    :return w:
    :return h:
    :return filt_sig:
    """
    # Define second-order sections representation of the IIR filter.
    sos = signal.butter(f_order, f_cut, 'lp', fs=f_sample, analog=False, output='sos')
    # Apply the filter to our signal.
    filt_sig = signal.sosfilt(sos, sig)
    
    if analysis_plot:
        # Compute the numerator and denominator polynomials of the IIR filter.
        b, a = signal.butter(f_order, f_cut, 'lp', fs=f_sample, analog=False)
        # Compute the frequency response of an analog filter.
        w, h = signal.freqs(b, a)
        # and plot results:
        t = np.linspace(0, n_samples - 1, n_samples)
        plot_wave_freqresp_filter(t, sig, filt_sig, w, h, f_order, f_cut)

    return filt_sig

### Time-Series Waveform Characterization: 
It does seem like part of the issue with the triggering logic is the lack of ability to compensate for what we will call "dead" channels or heavily distorted data channels.  These dead channels can be understood as having only static or noise oscillations. The distorted channels will be distorted square waves, and could be due to a number of factors including short-circuits in the signal receiver hardware and impedance mismatches in connections to the cable array and/or the topside control data circuitry. 

Accordingly, it makes sense to apply as efficient an automated characterization method as possible to see if there are dead or distorted channels that can be omitted from the analysis for now. 

In [9]:
# if the signals are below a certain threshold, they should be considered "dead"
# and used for measuring a baseline internal instrument noise metric only. 
def check_channel_activity(time_series_signal):
    pass

In [10]:
# if the signals are above a certain threshold, they could be 
# correlated with an ideal square wave by convolution? 
def check_channel_correlation(time_series_signal):
    pass

### Time Series Conditioning and Triggering Logic

In [11]:
def time_series_conditioning(raw_time_series_matrix, f_s, f_order, f_cut, analysis_plot):
    """
    .. fucntion:: signal_conditioning
    .. description::
    :param time_series_matrix:
    :return win_sig_0: Trigger-windowed ORIGINAL noisy signals (no filters) array with offset removed and start times (t0q) for channel-q shifted to match using triggers. 
    """
    # definitions
    # number of channels q
    q_channels = raw_time_series_matrix.shape[-1]
    
    # Remove DC offsets
    zeroed_time_series_matrix = remove_dc_offset(raw_time_series_matrix)
    
    # Locate trigger times and return arrays of t_starts and n_lengths. Include filter parameters.
    t_starts, n_lengths = set_triggers(zeroed_time_series_matrix, f_s, f_order, f_cut, analysis_plot)
    
    # set the minimum trimmed length to match all channels-q
    n_min = np.nanmin(n_lengths)
    # adjust the minimum length to n_min - 1 if the value for n_min is odd (for simplifying FFTs later)
    if n_min % 2 != 0:
        n_min = n_min - 1
    
    # define complex output signal array based on `n_min` and `q_channels`
    output_signal_matrix = np.empty((n_min, q_channels), dtype=np.complex64)
    
    # Iteratively Shift signals in each channel over and truncate to match the others 
    # based on the start triggers and array lengths determined by `set_triggers()`
    for q in range(0, q_channels):   
        if VERBOSE:
            print(f"Shifting signal start and trimming length for signal channel {q + 1}")
        output_signal_matrix[:, q] = shift_signal_to_triggers(raw_time_series_matrix[:, q], t_starts[q], n_min)
        
    # testprint
    if DEBUG:
        print(f"output_signal_matrix dimensions are now {output_signal_matrix.shape}...")
    return output_signal_matrix

In [12]:
def shift_signal_to_triggers(time_series_q, t0q, n_min):
    """
    .. function:: shift_signals_to_triggers
    .. description::
    :param time_series_q:
    :param t0q:
    :param n_min:
    :return channel_out:  Output array of length `n_min` for channel-q shifted to the global t0 and trimmed to match `n_min`.
    """
    if DEBUG:
        print(f"Rolling time-series by {-t0q} and truncating to length {n_min}...")
    return np.roll(time_series_q, -t0q)[0: n_min]

In [13]:
def set_triggers(s_t_matrix, f_s, filter_order, filter_cut, analysis_plot):
    """
    .. function: set_triggers:
    .. description::
    :param s_t_matrix:
    :param filter_order:
    :param filter_cut:
    :return t0s:
    :return nlengs:
    """
    # definitions:
    q_channels = s_t_matrix.shape[-1]
    n_raw = s_t_matrix.shape[0]
    t0s = []
    nlengs = []
    
    # for each channel-q:
    for q in range(0, q_channels):
        # filter
        filt_signal = butterworth_digital_lpf(s_t_matrix[:, q], n_raw, f_s, filter_order, filter_cut, analysis_plot)
        # compute gradient and max gradient value of filtered signal
        filt_g_signal = np.gradient(filt_signal)
        g_max = np.nanmax(filt_g_signal)
        n_filt = filt_signal.shape[0]
        # get first positive trigger t0q
        t0q = rising_edge_trigger(filt_signal, filt_g_signal, g_max, n_filt)
        # get last negative trigger tfq
        tfq = falling_edge_trigger(filt_signal, filt_g_signal, g_max, n_filt)
        t0s.append(t0q)
        nlengs.append(np.abs(tfq - t0q))
    return t0s, nlengs

In [14]:
def rising_edge_trigger(filt_zeroed_sig, filt_sig_gradient, gradient_max, N):
    """
    """
    # There is a problem with this logic... 
    positive_trigger_indices = (j for j in range(N - 1) if 
                           ((filt_sig_gradient[j] >= (2/3) * gradient_max) and
                            filt_zeroed_sig[j - 1] < filt_zeroed_sig[j]))
    t0 = next(positive_trigger_indices)
    if VERBOSE:
        print(f"First rising trigger found at index {t0}!")
    return t0

In [15]:
def falling_edge_trigger(filt_zeroed_sig, filt_sig_gradient, gradient_max, N):
    """
    """
    # There is a problem with this logic... 
    negative_trigger_indices = (j for j in range(N-1, 0, -1)  if 
                            ((filt_sig_gradient[j] <= (2/3) * -gradient_max) and
                             filt_zeroed_sig[j + 1] < filt_zeroed_sig[j]))
    tf = next(negative_trigger_indices)
    if VERBOSE:
        print(f"Last falling trigger found at index {tf}!")
    return tf

### Computing Apparent Impedance and Phase Shift Spectra

In [16]:
def apparent_impedance_spectrum(s0k, sqk):
    """
    Estimate the apparent impedance spectrum ||Z(k)||^2 for a shunt(s0_k) <--> RX Channel (sq_k) pair by taking the magnitude 
    of the difference between the two complex spectra. This yields the REAL part of the complex impedance of the target and water network
    between RX electrodes and removes the internal real impedance from the transmitter shunt. 
    
    :param s0k:
    :param sqk:
    :return zqk:
    """
    zqk = np.absolute(s0k - sqk)
    return zqk

In [17]:
def phase_difference_spectrum(s0k, sqk):
    """
    .. function:: phase_difference_spectrum()
    .. description:: Estimate the phase angle of each waveform and the associated shift between them, holding
    S_a(k) as a reference wavform and taking the difference of the derived phase arrays. 
    
    The phase shift corresponds to modulation of the complex part of the transmitted/
    received waveform, which is an indication of either inductive or capacitive frequency-
    dependent responses of the signal due to the electrical network betweewn the electrodes
    formed by the water and target. 
    
    Calculate the phase shift from a reference spectrum (sa_k) and a shifted spectrum (sb_k)
    
    :param s0k:
    :param sqk:
    :return phase_shift_spectrum:
    """
    phase0k = np.angle(s0k) 
    phaseqk = np.angle(sqk)
    phase_shift = np.subtract(phaseqk, phase0k)
    
    # Correct phase shift angles for values outside np.pi -- removes false positive/negative values.
    for m in range(phase_shift.shape[0]):
        if phase_shift[m] > np.pi:
            phase_shift[m] = 2.00 * np.pi - phase_shift[m]
        elif phase_shift[m] < -np.pi:
            phase_shift[m] = phase_shift[m] + 2.00 * np.pi
        else:
            continue
    
    return phase_shift

In [18]:
def packet_dsp(t, s_t, period, f_s, filter_order=4, filter_cut=200, filter_analysis_plot=False):
    """
    .. function:: 
    .. description:: Single-packet Signal Processing. 
    :param t:
    :param s_t:
    :param tau:
    :param filter_order:
    :param filter_cut:
    :return k_domain:
    :return conditioned_time_series:
    :return marine_ip_k:
    :return marine_r_k:
    """
    # def time_series_conditioning(raw_time_series_matrix, f_s, f_order, f_cut, analysis_plot
    conditioned_time_series = time_series_conditioning(s_t, f_s, filter_order, filter_cut, filter_analysis_plot)
    number_of_channels = conditioned_time_series.shape[-1]
    print("Now computing FFT, Marine Resistivity, and Marine IP Responses for the packet...")
    N_samples = conditioned_time_series.shape[0]
    # definitions for numpy arrays and constants
    tau = np.float(period / f_s)
    k_domain = fft.fftfreq(N_samples, tau)
    s_k = np.zeros((k_domain.shape[0], number_of_channels), dtype=np.complex64)
    marine_ip_k = np.zeros((k_domain.shape[0], number_of_channels), dtype=np.complex64)
    marine_r_k = np.zeros((k_domain.shape[0], number_of_channels), dtype=np.complex64)
    # iterate over all channels to compute the FFT, phase difference between shunt and channels, and apparent impedance spectra between channels
    for q in range(0, number_of_channels):
        s_k[:, q] = (2.00 / N_samples) * fft.fft(conditioned_time_series[:, q])
        marine_ip_k[:, q]= phase_difference_spectrum(s_k[:,0], s_k[:,q])
        marine_r_k[:, q] = apparent_impedance_spectrum(s_k[:, 0], s_k[:, q])
    if VERBOSE:
        print(f"Yielded two matrices: phase shift shape= ({marine_ip_k.shape}) and resistivity shape = ({marine_r_k.shape}))")
    return k_domain, conditioned_time_series, marine_ip_k, marine_r_k